## Installation

First, install java-8 and xvfb for running minerl:

In [ ]:
!apt-get -qq install openjdk-8-jdk xvfb > /dev/null

In [ ]:
!update-alternatives --config java

And make sure you are using Java 8:

In [ ]:
!java -version

Install the environment:

In [ ]:
# takes up to 4 minutes to finish
!pip install -q iglu==0.2.2

For now, just ignore version conflicts above.

In [ ]:
import gym
import iglu

The code below should be run ONLY in colab. For the first run, uncomment the first two bash commands. Consecutive runs don't require them.

In [ ]:
# exec this cell ONLY in colab
!wget -q https://raw.githubusercontent.com/iglu-contest/tutorials/main/env/colab_setup.sh -O - | sh > /dev/null 2>&1
!pip install -q pyvirtualdisplay
from pyvirtualdisplay import Display
disp = Display(backend="xvnc", size=(800, 600))
disp.start();
# for local notebooks instead launch jupyter as: xvfb-run -s "-screen 0 640x480x24" jupyter ...

Let's create the env and examine the action and observation spaces:

In [ ]:
env = gym.make('IGLUSilentBuilder-v0')

In [ ]:
env.action_space

In [ ]:
env.observation_space

In [ ]:
from tqdm import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output
from PIL import Image

At first run, the reset run minecraft and initializes the mission.

In [ ]:
obs = env.reset()

In [ ]:
display(Image.fromarray(obs['pov'], ))

In [ ]:
total_reward = 0.
done = False
while not done:
  display(Image.fromarray(obs['pov'],).resize((256, 256)), )
  action = env.action_space.sample()
  # action['camera'][0] *= 0.1
  # action['camera'][0] *= 0.3
  obs, reward, done, info = env.step(action)
  total_reward += reward
  clear_output(wait=True)

In [ ]:
total_reward

In [ ]:
obs['grid'].nonzero()

In [ ]:
len(obs['grid'].nonzero()[0])

In [ ]:
obs.keys()

In [ ]:
plt.imshow(obs['pov'])

In [ ]:
print(obs['chat'])

In [ ]:
{k: obs[k].shape for k in ['pov', 'agentPos', 'inventory', 'grid']}

At `'agentPos'` key, there are `x, y, z` coordinates and `pitch, yaw` angles.

In [ ]:
obs['agentPos']

`'inventory'` component 

In [ ]:
obs['inventory']

In [ ]:
info['target_grid'][:2, :4, :4]

Let's measure the speed:

In [ ]:
for ep in range(2):
  obs = env.reset()
  done = False
  with tqdm() as bar:
    while not done:
      bar.update(1)
      action = env.action_space.sample()
      obs, reward, done, info = env.step(action)

In the competition you are free to choose from three action spaces:

In [ ]:
gym.make('IGLUSilentBuilder-v0', action_space='discrete').action_space

In [ ]:
gym.make('IGLUSilentBuilder-v0', action_space='continuous').action_space

In [ ]:
gym.make('IGLUSilentBuilder-v0', action_space='human-level').action_space

In [ ]:
env.tasks

In [ ]:
obs = env.reset()

In [ ]:
env.tasks.current.chat

In [ ]:
from iglu.tasks import TaskSet

env.update_taskset(TaskSet(preset=['C1', 'C2', 'C3']))
env.tasks

In [ ]:
import numpy as np
from iglu.tasks import CustomTasks

custom_grid = np.zeros((9, 11, 11)) # (y, x, z)
custom_grid[:3, 5, 5] = 1 # blue color
env.update_taskset(CustomTasks(
[
    ('<Architect> Please, build a stack of three blue blocks somewhere.\n'
     '<Builder> Sure.',
     custom_grid)
]
))
obs = env.reset()

In [ ]:
print(env.tasks.current.chat)

In [ ]:
from iglu.tasks import RandomTasks

env.update_taskset(RandomTasks(max_blocks=3, max_dist=5, num_colors=3))

In [ ]:
env.tasks

In [ ]:
env.tasks.sample().target_grid.nonzero()

In [ ]:
env.close()